In [215]:
# note: latest method using pymatgen CN model 


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import rc, font_manager
ticks_font = font_manager.FontProperties(family='serif', style='normal',
                                         size=24, weight='normal', stretch='normal')

from pymatgen.core import Structure, Site,PeriodicSite,Lattice,Species,IStructure
import os
from pymatgen.io.cif import CifFile, CifParser, CifWriter
from pymatgen.io.vasp import *
import crystal_toolkit
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import alphashape
from scipy.spatial import ConvexHull, Delaunay
from pymatgen.analysis.structure_matcher import StructureMatcher
import spglib


In [369]:
from copy import deepcopy

In [3]:
from pymatgen.analysis.molecule_structure_comparator import CovalentRadius

In [69]:
%load_ext autoreload
%autoreload 2
from kagome_tools import *
import ast
import plotly.graph_objects as go

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from itertools import permutations
import itertools

In [8]:
import random
import importlib
from tqdm.notebook import tqdm, trange
import time 


In [9]:
import ast

In [10]:
from pymatgen.analysis.chemenv.utils.coordination_geometry_utils import Plane

In [11]:
spg_data = [[],[]]
spg_dict = {}
for i in range(530):
    i = i+1
    spg_ = spglib.get_spacegroup_type(i)
    if spg_['number'] not in spg_data[0]:
        spg_data[0].append(spg_['number'])
        spg_data[1].append(spg_['international_short'])
        spg_dict[spg_['number']] = spg_['international_short']

In [12]:
# cif file locations
top_dir = '../../intermetallics_search_results2/'

In [13]:
all_files = os.listdir(top_dir)

In [14]:
# gph file locations
gph_top = '../Intermetallics_kagome_search_graph/'
all_gph = os.listdir(gph_top)

In [15]:
cifs = []
gphs = []
cifs_ids = []
cifs_ids_short = []
for f in all_files:
    
    cifs.append(f)
    cifs_ids.append((f.split('_')[-1].split('.')[0]))
    cifs_ids_short.append( int(f.split('_')[-1].split('.')[0]))
for f in all_gph: 

    gphs.append(f)



In [16]:
sorted_cifs = [z for x,y,z in sorted(zip(cifs_ids_short,cifs_ids, cifs))]
sorted_cifs_ids = [y for x,y,z in sorted(zip(cifs_ids_short,cifs_ids, cifs))]
sorted_cifs_ids_short = [x for x,y,z in sorted(zip(cifs_ids_short,cifs_ids, cifs))]

In [17]:
def transmat(lattice_A,lattice_B):
    # transformation from lattice A to lattice B
    M_A_B = np.dot(lattice_B, np.linalg.inv(lattice_A))
    return M_A_B

In [18]:
def clean_file(gphfile):
    with open(gphfile) as fh:
        lines = fh.readlines()
        
    vec_info = np.array([np.float64(x) for x in lines[1].strip('\n').split(' ')])
    #print(vec_info)
    site_list = []
    lattice = Lattice.from_parameters(a=vec_info[0],b=vec_info[1],c=vec_info[2],alpha=vec_info[3],beta=vec_info[4],gamma=vec_info[5])
    for line in lines[2:]:
        
        ls = line.strip('\n').split(' ')
        
        coor_txt = [ls[i+2] for i in range(3)]
        ele = Species(ls[1])
        coor = np.array([np.float64(x) for x in coor_txt])
        
        site = PeriodicSite(ele,coords=coor,lattice=lattice)
        site_list.append(site)
    structure = Structure.from_sites(site_list)
    return structure

In [19]:
structure_sub = clean_file(gph_top+gphs[0])
#structure

In [20]:
len(cifs)

43170

In [21]:
def get_site_idx(structure_sub,structure_full):
    site_sub = structure_sub.sites
    site_full = structure_full.sites
    
    sub_full_ids = []
    for i in range(len(site_sub)):
        site_i = site_sub[i]
        for j in range(len(site_full)):
            site_j = site_full[j]
            dist = np.linalg.norm(site_i.coords-site_j.coords)
            if np.round(dist,1)==0:
                #print(dist, site_i,site_j)
                sub_full_ids.append(j)
                
    return sub_full_ids

In [22]:
data = pd.read_csv('./bulk_1_all.csv')


In [25]:
data['cifs_name'] = data['sorted_cifs_non_empty']

In [26]:
gph_write_cif_dir = '../Intermetallics_kagome_search_graph_gph_to_cifs/'

In [164]:
# start looping to get the structures stored in a list: 
# very simple one, just compare the sites 
miller_index_list = []
structure_full_list = []
structure_full_index_list = []
structure_sub_list = []
structure_full_n_atom_list = []
spg_number_list = []

p_bar = tqdm(range(len(data))[:])
#for i in range(len(sorted_cifs_ids_short)):
for i in p_bar:
    p_bar.set_description(f'Working on "{i}"')
    cif_file = data['cifs_name'][i]
    try:
        cif = CifParser(top_dir+cif_file,occupancy_tolerance=100,site_tolerance=0.008)
        structure_full_orig = cif.get_structures(primitive=False)[0]
    except:
        cif = CifParser(top_dir+cif_file,occupancy_tolerance=101,site_tolerance=0.008)
        structure_full_orig = cif.get_structures(primitive=False)[0]
    
    structure_full = structure_full_orig
    structure_sub_ = []
    gph_list = ast.literal_eval(data['gph_grouped_non_empty'][i])
    for j in range(len(gph_list)):
        
        gph_fn = gph_list[j]
        structure_sub = clean_file(gph_top+gph_fn)
        #
        new_structure_sub = Structure.from_sites(structure_sub.sites)
        structure_sub_.append(structure_sub)
        
        #cw = CifWriter(structure_sub)
        #cw.write_file(gph_write_cif_dir+'{}_{}.cif'.format(sorted_cifs_ids_non_empty[i],j))
        
    structure_full_n_atom_list.append(len(structure_full.sites))
    structure_full_list.append(structure_full)
    structure_sub_list.append(structure_sub_)
    
    cif_dict = cif.as_dict()
    key=list(cif_dict.keys())[0]
    spg_number_list.append(int(cif_dict[key]['_space_group_IT_number']))

  0%|          | 0/15000 [00:00<?, ?it/s]

In [168]:
compositions = []
for i in range(len(structure_full_list)):
    structure_full = structure_full_list[i]
    comp = structure_full.composition.as_dict()
    compositions.append(comp)

In [29]:
all_jimages = get_all_images()


In [84]:
data['num_atom'] = structure_full_n_atom_list

In [166]:
data['space_group_num'] = spg_number_list

In [427]:
data['composition'] = compositions

In [30]:
unique_gph_all = []
p_bar = tqdm(data.index)
#p_bar = tqdm([5])
for i in p_bar:
    #time.sleep(0.5)
    p_bar.set_description(f'Working on "{i}"')
    if len(data['gph_grouped_non_empty'][i])>1:
        structure_sub = structure_sub_list[i][0]
        #A,B,C,D,s = get_plane_from_sites_m(structure_sub.sites)
        A,B,C,D = get_plane_from_sites_m3(structure_sub.sites)
        plane_my = Plane([A,B,C,-D])
        unique_gph = [0]
        gph_planes = [plane_my]
        
        for jt in range(len(structure_sub_list[i])-1):
            j = jt+1
            structure_sub = structure_sub_list[i][j]
            points = [s.coords for s in structure_sub]
            distances_ = []
            for plane_known in gph_planes:
                distances = plane_known.distances_indices_groups(points,0.05)
                distances_.append(np.abs(np.mean(distances[0])))
            #print(distances_)
            if min(distances_) > 0.15:
                #print(np.round(distances[0],2))

                #A,B,C,D,s = get_plane_from_sites_m(structure_sub.sites)
                A,B,C,D = get_plane_from_sites_m3(structure_sub.sites)
                plane_my = Plane([A,B,C,-D])
                gph_planes.append(plane_my)
                unique_gph.append(j)
    else:
        unique_gph = [0]
    unique_gph_all.append(unique_gph)
#data['unique_gph']=unique_gph_all



  0%|          | 0/15000 [00:00<?, ?it/s]

In [171]:
data['gph_grouped'] = data['gph_grouped_non_empty']

In [177]:
data['unique_gph'] = unique_gph_all

In [89]:
# without 3 atoms in the cell, it is not possible to construct the lattice plane from 3 sites

filtered_3_remain_index = [x for x in data.index if data['num_atom'][x]>=3]
len(filtered_3_remain_index)


13492

In [704]:
#filtered_3_remain_index = list(filtered_3_data.index)

# filter round 4
# build a planar structure based on the sites. 
filter_4_match_res = [] # document the ABCD of that plane
plane_structures_list = []
plane_structures_pbc_list = []
p_bar = tqdm(data.index)
for i in p_bar:
    p_bar.set_description(f'Working on "{i}"')
    if i in filtered_3_remain_index[:]:
        structure_full = structure_full_list[i]        
        pop_sites = populate_sites(structure_full,all_jimages)
        match_fragment_plane = [] # document the ABCD of that plane 
        plane_structures_ = []
        plane_structures_pbc_ = []
        #p_bar2 = tqdm(range(len(ast.literal_eval(data['unique_gph'][i]))))
        for j in range(len((data['unique_gph'][i]))):
        #for j in range(len(data['unique_gph'][i])):
            #p_bar2.set_description(f'Working on substructure"{j}"')
            jt = data['unique_gph'][i][j]
            structure_sub = structure_sub_list[i][jt]
            A,B,C,D = get_plane_from_sites_m3(structure_sub.sites)
            # get_plane_from_sites_m is not good enough
            plane_my = Plane([A,B,C,-D])
            #for site in structure_sub.sites:
            #    print(calc_site_plane_distance(A,B,C,D,site))
            sites_in_plane,sites_plane_dist = get_sites_in_plane(structure_full,pop_sites,A,B,C,D)
            plane_sites = [structure_full.sites[x] for x in sites_in_plane]
            sites_in_gph, sites_out_gph = get_site_in_out_gph(plane_sites,structure_sub)
            # group atoms in the supercell based on the distance
            points = [s.coords for s in plane_sites]
            plane_dist_site_group = plane_my.distances_indices_groups(points,0.15)
            # this gives us the sites with approximately the same distances to the plane
            # so that I know these sites should be on the gph, but gph may exclude those sites
            
            
            #print(np.round(plane_dist_site_group[0],2),plane_dist_site_group[2])
            if len(plane_dist_site_group[2])>1:
                num_sites_group = [(len(x)) for x in plane_dist_site_group[2]]
                more_sites_group = num_sites_group.index(max(num_sites_group))
                plane_sites_clean = [plane_sites[int(i)] for i in plane_dist_site_group[2][more_sites_group]]
            else:
                plane_sites_clean = plane_sites
            # get plane_sites
            # get A,B,C,D
            if len(plane_sites_clean)<3:
                #number of atoms <3
                nA,nB,nC,nD,nplane_sites = [A,B,C,D,plane_sites]
            elif len(plane_sites_clean)==3:
                if points_collinear(plane_sites_clean):
                    nA,nB,nC,nD,nplane_sites = [A,B,C,D,plane_sites]
                else:
                    # number of atoms ==3
                    nA,nB,nC,nD,nplane_sites = get_plane_from_sites(plane_sites_clean)
            else:
                # number of atoms > 3
                nA,nB,nC,nD = get_plane_from_sites_m3(plane_sites_clean)
            #match_fragment_plane.append([nA,nB,nC,nD]) 
            # make a cleaner plane_structure, this is crucial to further steps. 
            # 1. check if other sites are missed on the facets/edges 
            plane_sites_clean_ = [s for s in plane_sites_clean]
            if len(plane_sites_clean)<len(plane_sites):
                for site in plane_sites:
                    if site not in plane_sites_clean_:
                        if 0 in [np.round(x,1) for x in site.frac_coords]:
                            plane_sites_clean_.append(site)
                        elif 1 in [np.round(x,1) for x in site.frac_coords]:
                            plane_sites_clean_.append(site)
            # 2. add atoms if needed  
            plane_func = Plane([nA,nB,nC,-nD])
            if len(plane_sites_clean)<len(plane_sites_clean_):
                plane_sites_cleaned,plane_sites_pbc = get_plane_sites_clean(plane_sites,plane_sites_clean,all_jimages,plane_func)
                if len(plane_sites_cleaned)>3:
                    nA,nB,nC,nD = get_plane_from_sites_m3(plane_sites_cleaned)
            else:
                #plane_sites_cleaned = get_plane_sites_clean_m2(plane_sites_clean)
                plane_sites_cleaned,plane_sites_pbc = get_plane_sites_clean(plane_sites,plane_sites_clean,all_jimages,plane_func)
                if len(plane_sites_cleaned)>3:
                    nA,nB,nC,nD = get_plane_from_sites_m3(plane_sites_cleaned)
            # get plane structure
            plane_structure = Structure.from_sites(plane_sites_cleaned)
            plane_structures_.append(plane_structure)
            # get plane structure with PBC 
            plane_structure_pbc = Structure.from_sites(plane_sites_pbc)
            plane_structures_pbc_.append(plane_structure_pbc)
            # get plane function 
            match_fragment_plane.append([nA,nB,nC,nD]) 
        filter_4_match_res.append(match_fragment_plane)
        plane_structures_list.append(plane_structures_)
        plane_structures_pbc_list.append(plane_structures_pbc_)
    else:
        plane_structures_pbc_list.append([])
        filter_4_match_res.append([])
        plane_structures_list.append([])

  0%|          | 0/15000 [00:00<?, ?it/s]

In [705]:
check_ = []
for i in p_bar:
    p_bar.set_description(f'Working on "{i}"')
    if i in filtered_3_remain_index[:]:
        for j in range(len((data['unique_gph'][i]))):
            jt = data['unique_gph'][i][j]
            if j!=jt:
                check_.append([i,j])

In [663]:
range(len(data['unique_gph'][i])),len(structure_sub_list[i])

150

In [725]:
data['match_gph'] = filter_4_match_res


In [97]:
#plane_func.distances_indices_groups([x.coords for x in plane_structure.sites],0.2)

In [103]:

def safe_literal_eval(s):
    try:
        # Handle infinity and negative infinity cases
        s = s.replace("inf", "0").replace("-inf", "0").replace("nan","0")
        
        return ast.literal_eval(s)
    except ValueError:
        # Handle other potential exceptions, if needed
        return s



[0]


In [104]:
#res = safe_literal_eval(data["match_gph"][8712])
#res

In [255]:
# write out the unique plane structure into cifs 
#
'''
plane_structures_write_dir = './plane_structures/'
plane_structures_pbc_write_dir = './plane_structures_pbc/'
p_bar = tqdm(data.index[:])
for i in p_bar:
    p_bar.set_description(f'Working on "{i}"')
    if i in filtered_3_remain_index[:]:
        CN_ = []
        cutoff_ = []
        # replace this line if necessary
        #absds = safe_literal_eval(data['match_gph'][i])
        absds = data['match_gph'][i]
        for j in data['unique_gph'][i]:
            #for j in [1,2]:
            
            jt = data['unique_gph'][i].index(j)
            #plane_structure = plane_structures_list[i][jt]
            plane_structure = plane_structures_list[i][jt]
            plane_structure_pbc = plane_structures_pbc_list[i][jt]
            
            cw1 = CifWriter(plane_structure)
            cifname = data['cifs_name'][i].split('_')[-1].strip('.cif')
            path_1 = plane_structures_write_dir + f'{cifname}_{jt}_{j}.cif'
            #print(path_1)
            cw1.write_file(path_1)
            
            cw2 = CifWriter(plane_structure_pbc)
            cifname = data['cifs_name'][i].split('_')[-1].strip('.cif')
            path_2 = plane_structures_pbc_write_dir + f'{cifname}_{jt}_{j}.cif'
            #print(path_2)
            cw2.write_file(path_2)
            
            

  0%|          | 0/15000 [00:00<?, ?it/s]

In [728]:

CN_f_4 = []
p_bar = tqdm(data.index)
cutoff_that_worked = []
for i in p_bar:
    p_bar.set_description(f'Working on "{i}"')
    if i in filtered_3_remain_index[:]:
        CN_ = []
        cutoff_ = []
        # replace this line if necessary
        #absds = safe_literal_eval(data['match_gph'][i])
        absds = data['match_gph'][i]
        for j in data['unique_gph'][i]:
            #for j in [1,2]:
            
            jt = data['unique_gph'][i].index(j)
            #plane_structure = plane_structures_list[i][jt]
            plane_structure = plane_structures_list[i][jt]
            plane_structure_pbc = plane_structures_pbc_list[i][jt]
            
            [A,B,C,D] = absds[jt]
            # analyze nninfo and get CN
            #nn_sites,CN = get_1nn_on_plane(plane_structure,plane_fun_coeff=plane_fun_coeff)
            for cutoff in range(7,30):
                try:
                    # Try to call your function with the current parameter
                    C_NN = CrystalNN(distance_cutoffs=(2,2.4),x_diff_weight=0.0, porous_adjustment=False,search_cutoff=cutoff )
                    nn_sites,nn_info = get_1nn(plane_structure_pbc,
                                       plane_structure,
                                       [A,B,C,D],
                                       NN_method=C_NN
                                      )
                    cutoff_.append(cutoff)
                    break
                except Exception as e:
                    # If there is an error, print it and continue to the next parameter
                    continue
            
            keys = nn_info.keys()
            #for j in range(len(nn_info["0"]['nn_info'])):
            #    print(plane_handle.distances_indices_groups([nn_info["0"]['nn_info'][j]['site'].coords])[0])
            CN = [len(nn_info[k]['nn_info']) for k in keys]
            CN_.append(CN)
        CN_f_4.append(CN_)
        cutoff_that_worked.append(cutoff_)
    else:
        CN_f_4.append([])
        cutoff_that_worked.append([])

  0%|          | 0/15000 [00:00<?, ?it/s]

In [733]:
data['CN_plane'] = CN_f_4

In [734]:
# Group_1: CN=4
# Group_2: CN=6
# Group_3: CN=8
# Group_4: CN=12
# Group_5: other cases

G_all = []
p_bar = tqdm(data.index)
for i in p_bar:
    time.sleep(1/1000)
    p_bar.set_description(f'Working on "{i}"')
    G_i_ = []
    if i in filtered_3_remain_index[:]:
        CN_list = data['CN_plane'][i]
        #print(np.unique(CN_list))
        CN_ = np.array([np.unique([y for y in x if y!=0]) for x in CN_list])
        #print(CN_.shape[1]==1)
        for CN_i in CN_:
            if (4 in CN_i)and(len(CN_i)==1):
                G_i = 1
            elif (6 in CN_i)and(len(CN_i)==1):
                G_i = 2
            elif (8 in CN_i)and(len(CN_i)==1):
                G_i = 3
                #print(i)
            elif (12 in CN_i)and(len(CN_i)==1):
                G_i = 4
                #print(i)
            else:
                G_i = 5
                #print(i,CN_i)
            G_i_.append(G_i)
    G_all.append(G_i_)

  0%|          | 0/15000 [00:00<?, ?it/s]

In [735]:
G_all[4484]

[2, 2, 2, 2]

In [736]:
data['CN_group_index'] = G_all

In [737]:
G_1_index = []

for i in data.index[:]:
    if 1 in data['CN_group_index'][i]:
        G_1_index.append(i)
        if 2 in data['CN_group_index'][i]:
            pass
            #print('G2',i)
        elif 3 in data['CN_group_index'][i]:
            pass
            #print('G3',i)
        elif 4 in data['CN_group_index'][i]:
            pass
            #print('G4',i)
        elif 5 in data['CN_group_index'][i]:
            pass
            #print('G5',i)
        

In [740]:
# analyze G-1 for inter-atomic distances and angles
G_1_angles = []
G_1_angle_distance_info_all = []
#data_all.iloc[G_1_index].index[:5]
p_bar = tqdm(data.index)
for i in p_bar:
    if i in G_1_index:
        p_bar.set_description(f'Working on "{i}"')
        # replace the following line if necessary 
        #absds = safe_literal_eval(data['match_gph'][i])
        #
        absds = data['match_gph'][i]
        ####
        # here we analyze all the gph if the corresponding CN_g_index is 1 
        angles_gph_1 = []
        distance_gph_1 = []
        for jt in range(len(data['CN_group_index'][i])):
            jidx = data['CN_group_index'][i][jt]
            if jidx==1:
                #plane_structure = plane_structures_list[i][jt]
                plane_structure = plane_structures_list[i][jt]
                plane_structure_pbc = plane_structures_pbc_list[i][jt]

                [A,B,C,D] = absds[jt]
                # analyze nninfo and get CN
                #nn_sites,CN = get_1nn_on_plane(plane_structure,plane_fun_coeff=plane_fun_coeff)
                for cutoff in range(7,30):
                    try:
                        # Try to call your function with the current parameter
                        C_NN = CrystalNN(distance_cutoffs=(2,2.4),x_diff_weight=0.0, porous_adjustment=False,search_cutoff=cutoff )
                        nn_sites,nn_info = get_1nn_m(plane_structure_pbc,
                                           plane_structure,
                                           [A,B,C,D],
                                           NN_method=C_NN
                                          )
                        cutoff_.append(cutoff)
                        break
                    except Exception as e:
                        # If there is an error, print it and continue to the next parameter
                        continue

                keys = nn_info.keys()
                # angles are rounded to 0.001
                angles_gph_1.append(np.unique(np.concatenate([[x[3] for x in nn_info[k]['angle_info']]  for k in keys])))
                distance_gph_1.append([ nn_info[k]['angle_info'] for k in keys])
        G_1_angles.append(angles_gph_1)
        G_1_angle_distance_info_all.append(distance_gph_1)

  0%|          | 0/15000 [00:00<?, ?it/s]

In [748]:
def angle_outside_range(angles):
    '''
    angle in degree, 
    outside +/- 5 from 60 and 120 will be reported as True
    '''
    x = [True for angle in angles if (0<= angle and angle<55)or(65< angle and angle<115)or(125< angle and angle<175)]
    if True in x:
        return True
    else:
        return False

In [751]:
G_1_angle_reg_index = []
G_1_angle_nonreg_index = []
G_1_angle_nonreg_angle = []


for i, angle_list in zip(G_1_index,G_1_angles):
    reg_ang_true = False
    for angle in angle_list:
        if len(angle)==2 and 60 in angle and 120 in angle:
            reg_ang_true = True
            
    if reg_ang_true:
        G_1_angle_reg_index.append(i)
    else:
        G_1_angle_nonreg_angle.append(angle)
        G_1_angle_nonreg_index.append(i)

In [752]:
len(G_1_angle_reg_index),G_1_angle_reg_index[:5]

(6734, [1, 3, 5, 8, 11])

In [753]:
#plane_structures_list[5553][1]

In [754]:
len(G_1_angle_reg_index),G_1_angle_reg_index[:5]

(6734, [1, 3, 5, 8, 11])

In [755]:
all_data_G_1_angle_labels = []
all_data_G_1_angle_data = [] 
# all empty nets
# empty net with reasonable angles: 1
# empty net with not reasonable angles: 2
# non-empty net: 3
for i in data.index:
    if i in G_1_index:
        sub_labels = []
        sub_data = []
        angle_list = G_1_angles[G_1_index.index(i)]
        for angle in angle_list:
            sub_data.append(list(angle))
            if len(angle)==2 and 60 in angle and 120 in angle:
                sub_labels.append(1)
            else:
                sub_labels.append(2)
        all_data_G_1_angle_labels.append(sub_labels)
        all_data_G_1_angle_data.append(sub_data)
    else:
        all_data_G_1_angle_labels.append([3])
        all_data_G_1_angle_data.append('N/A')

In [756]:
data['angle_labels']=all_data_G_1_angle_labels
data['unique_angles']=all_data_G_1_angle_data

In [757]:
#data.rename(columns={"angle_data": "unique_angles"})

In [759]:
#G_1_angle_distance_info_all[G_1_index.index(16)]

In [761]:
#G_1_angle_nonreg_angle

In [762]:
#plane_structures_pbc_list[1400][0]


In [763]:
# define functions to analyze std of angles around an atom and atomic distances
def quantity_std(site_info):
    '''
    site_info: a list of sites information containing atom index, angles, and distances
    site_info[i][ii]=[j,i,k,angle,dij,dik,djk] 
    return: site grouped angles and distances
    '''
    l_i = range(len(site_info))
    all_si_di = []
    all_si_angles = []
    for i in l_i:
        si = np.array(site_info[i])
        si_angles = si.T[3]
        jk = [] # site-j and site-k index
        si_di = [] # distance around site-i
        for j in range(len(si.T[0])):
            j_a = si.T[0][j]
            if j_a not in jk:
                jk.append(j_a)
                si_di.append(si.T[4][j])
        for k in range(len(si.T[2])):
            k_a = si.T[2][k]
            if k_a not in jk:
                jk.append(k_a)
                si_di.append(si.T[4][k])
        #print(jk)
        all_si_di.append(np.array(si_di))
        all_si_angles.append(si_angles)
    all_si_di = np.array(all_si_di)
    all_si_angles = np.array(all_si_angles)
    return all_si_di,all_si_angles

# define functions to get the angles and distances around specified sites-i
def get_angles_distances_jik(site_info):
    '''
    site_info: a list of sites information containing atom index, angles, and distances
    site_info[i][ii]=[j,i,k,angle,dij,dik,djk] 
    return: site grouped Angles_jik and Distances-djk
    '''
    l_i = range(len(site_info))
    all_si_di = []
    all_si_angles = []
    for i in l_i:
        si = np.array(site_info[i])
        si_angles = si.T[3]
        si_djk = si.T[-1]
        
        all_si_di.append(np.array(si_djk))
        all_si_angles.append(si_angles)
    all_si_di = np.array(all_si_di)
    all_si_angles = np.array(all_si_angles)
    return all_si_di,all_si_angles


In [764]:
# function to group the nearest sites according to angles from get_angles_distances_jik
def get_quantity_std(all_site_distances,all_site_angles,angles_around=60,delta=40):
    '''
    all_site_distances,all_site_angles: np.arrays from get_angles_distances_jik
    angels_around: only return distances when the angles are roughtly equal to this value.
    delta: difference between max and min angles for grouping the angles. 
    '''
    l_i = range(len(all_site_distances))
    selected_angles = np.array([])
    selected_djk = np.array([])
    for i in l_i:
        #print(selected_angles)
        si_angles = all_site_angles[i]#-angles_around
        si_angles_abs = np.abs(all_site_angles[i]-angles_around)
        si_djk = all_site_distances[i]
        
        angles_abs_sort,angles_abs_group,angles_abs_group_index = group_angles(si_angles_abs,delta=delta)
        #print(np.array([si_angles[int(index)] for index in angles_abs_group_index[0]]))
        selected_angles = np.concatenate([selected_angles,np.array([si_angles[int(index)] for index in angles_abs_group_index[0]])])
        selected_djk = np.concatenate([selected_djk,np.array([si_djk[int(index)] for index in angles_abs_group_index[0]])])
    return selected_angles,selected_djk

In [770]:
all_data_angles_around_60_mean = []
all_data_angles_around_60_std = []
all_data_djk_around_60_mean = []
all_data_djk_around_60_std = []
all_data_angles_around_60 = []
all_data_djk_around_60 = []

# no data: 'NA'
for i in data.index:
    if i in G_1_index:
        angles_std = []
        angles_mean = []
        djk_std = []
        djk_mean = []
        all_angles_jik_list = []
        all_djk_list = []
        iG = G_1_index.index(i)
        for j in range(len(G_1_angle_distance_info_all[iG])):
            G_1_angle_distance_info_ = G_1_angle_distance_info_all[iG][j]
            
            test_di,test_angles = get_angles_distances_jik(G_1_angle_distance_info_)
            all_angles_jik, all_djk = get_quantity_std(test_di,test_angles,angles_around=60,delta=20)
            
            djk_std.append(np.round(np.std(all_djk),5))
            angles_std.append(np.round(np.std(all_angles_jik),5))
            djk_mean.append(np.round(np.mean(all_djk),5))
            angles_mean.append(np.round(np.mean(all_angles_jik),5))
            all_angles_jik_list.append(list(all_angles_jik))
            all_djk_list.append(list(all_djk))
            
        all_data_djk_around_60_std.append(djk_std)
        all_data_angles_around_60_std.append(angles_std)
        all_data_djk_around_60_mean.append(djk_mean)
        all_data_angles_around_60_mean.append(angles_mean)
        all_data_angles_around_60.append(all_angles_jik_list)
        all_data_djk_around_60.append(all_djk_list)
    else:
        all_data_djk_around_60_std.append('N/A')
        all_data_angles_around_60_std.append('N/A')
        all_data_djk_around_60_mean.append('N/A')
        all_data_angles_around_60_mean.append('N/A')
        all_data_angles_around_60.append('N/A')
        all_data_djk_around_60.append('N/A')

In [771]:
data['distance_mean_60'] = all_data_djk_around_60_mean
data['distance_std_60'] = all_data_djk_around_60_std
data['angles_mean_60'] = all_data_angles_around_60_mean
data['angles_std_60'] = all_data_angles_around_60_std
data['angles_60_data'] = all_data_angles_around_60
data['distance_60_data'] = all_data_djk_around_60

In [772]:
all_data_angles_around_120_mean = []
all_data_angles_around_120_std = []
all_data_djk_around_120_mean = []
all_data_djk_around_120_std = []
all_data_angles_around_120 = []
all_data_djk_around_120 = []

# no data: 'NA'
for i in data.index:
    if i in G_1_index:
        angles_std = []
        angles_mean = []
        djk_std = []
        djk_mean = []
        all_angles_jik_list = []
        all_djk_list = []
        iG = G_1_index.index(i)
        for j in range(len(G_1_angle_distance_info_all[iG])):
            G_1_angle_distance_info_ = G_1_angle_distance_info_all[iG][j]
            
            test_di,test_angles = get_angles_distances_jik(G_1_angle_distance_info_)
            all_angles_jik, all_djk = get_quantity_std(test_di,test_angles,angles_around=120,delta=20)
            
            djk_std.append(np.round(np.std(all_djk),5))
            angles_std.append(np.round(np.std(all_angles_jik),5))
            djk_mean.append(np.round(np.mean(all_djk),5))
            angles_mean.append(np.round(np.mean(all_angles_jik),5))
            all_angles_jik_list.append(list(all_angles_jik))
            all_djk_list.append(list(all_djk))
            
        all_data_djk_around_120_std.append(djk_std)
        all_data_angles_around_120_std.append(angles_std)
        all_data_djk_around_120_mean.append(djk_mean)
        all_data_angles_around_120_mean.append(angles_mean)
        all_data_angles_around_120.append(all_angles_jik_list)
        all_data_djk_around_120.append(all_djk_list)
    else:
        all_data_djk_around_120_std.append('N/A')
        all_data_angles_around_120_std.append('N/A')
        all_data_djk_around_120_mean.append('N/A')
        all_data_angles_around_120_mean.append('N/A')
        all_data_angles_around_120.append('N/A')
        all_data_djk_around_120.append('N/A')

In [773]:
data['distance_mean_120'] = all_data_djk_around_120_mean
data['distance_std_120'] = all_data_djk_around_120_std
data['angles_mean_120'] = all_data_angles_around_120_mean
data['angles_std_120'] = all_data_angles_around_120_std
data['angles_120_data'] = all_data_angles_around_120
data['distance_120_data'] = all_data_djk_around_120

In [774]:
data.columns

Index(['Unnamed: 0', 'sorted_cifs_ids_non_empty',
       'sorted_cifs_ids_short_non_empty', 'gph_grouped_non_empty',
       'sorted_cifs_non_empty', 'cifs_name', 'unique_gph', 'num_atom',
       'match_gph', 'CN_plane', 'CN_group_index', 'angle_labels',
       'unique_angles', 'distance_mean_60', 'distance_std_60',
       'angles_mean_60', 'angles_std_60', 'angles_60_data', 'distance_60_data',
       'distance_mean_120', 'distance_std_120', 'angles_mean_120',
       'angles_std_120', 'angles_120_data', 'distance_120_data', 'cifs_ids',
       'space_group_num', 'composition', 'gph_grouped'],
      dtype='object')

In [776]:
data

,Unnamed: 0,sorted_cifs_ids_non_empty,sorted_cifs_ids_short_non_empty,gph_grouped_non_empty,sorted_cifs_non_empty,cifs_name,unique_gph,num_atom,match_gph,CN_plane,...,distance_mean_120,distance_std_120,angles_mean_120,angles_std_120,angles_120_data,distance_120_data,cifs_ids,space_group_num,composition,gph_grouped
0,0,71,71,"['Fe3Sn2-71-Cagome_big.gph', 'Fe3Sn2-71-Cagome...",intermetallics_search_000071.cif,intermetallics_search_000071.cif,[0],30,"[[-0.10606060825053208, -0.06123412072386011, ...","[[6, 6, 6, 6]]",...,N/A,N/A,N/A,N/A,N/A,N/A,intermetallics_search_000071.cif,166,"{'Fe0+': 18.0, 'Sn0+': 12.0}","['Fe3Sn2-71-Cagome_big.gph', 'Fe3Sn2-71-Cagome..."
1,1,74,74,"['Fe3Mn4Ge6-74-Cagome_big.gph', 'Fe3Mn4Ge6-74-...",intermetallics_search_000074.cif,intermetallics_search_000074.cif,[0],13,"[[9.84556895428385e-16, 1.7053025658242402e-15...","[[4, 4, 4]]",...,[4.43405],[0.0],[120.0],[0.0],"[[120.0, 120.0, 120.0, 120.0, 120.0, 120.0]]","[[4.43405, 4.43405, 4.43405, 4.43405, 4.43405,...",intermetallics_search_000074.cif,191,"{'Fe0+': 3.0001999999999995, 'Mn0+': 3.9998000...","['Fe3Mn4Ge6-74-Cagome_big.gph', 'Fe3Mn4Ge6-74-..."
2,2,77,77,"['Bi3Y5-77-Cagome_big.gph', 'Bi3Y5-77-Cagome_b...",intermetallics_search_000077.cif,intermetallics_search_000077.cif,[0],32,"[[0.0, -0.3788635297973532, -0.0, -0.890424010...","[[5, 5, 6, 6, 5, 5, 4, 4]]",...,N/A,N/A,N/A,N/A,N/A,N/A,intermetallics_search_000077.cif,62,"{'Y0+': 20.0, 'Bi0+': 12.0}","['Bi3Y5-77-Cagome_big.gph', 'Bi3Y5-77-Cagome_b..."
3,3,362,362,"['Cs6K7-362-Cagome_big.gph', 'Cs6K7-362-Cagome...",intermetallics_search_000362.cif,intermetallics_search_000362.cif,[0],26,"[[-1.7456666904760174e-15, -1.0078584460337928...","[[4, 4, 4]]",...,[7.86178],[0.0],[120.0],[0.0],"[[120.0, 120.0, 120.0, 120.0, 120.0, 120.0]]","[[7.86178, 7.86178, 7.86178, 7.86178, 7.86178,...",intermetallics_search_000362.cif,194,"{'Cs0+': 12.0, 'K0+': 14.0}","['Cs6K7-362-Cagome_big.gph', 'Cs6K7-362-Cagome..."
4,4,458,458,"['Tb2(Fe0.832Al0.168)17-458-Cagome_big.gph', '...",intermetallics_search_000458.cif,intermetallics_search_000458.cif,[0],22,"[[-1.5820614745242897e-17, -2.7402108545734038...","[[4, 4, 4, 4, 4, 4, 4, 4, 4]]",...,[4.266],[0.0007],[120.0],[0.01633],"[[120.0, 120.02, 120.0, 120.02, 120.0, 120.02,...","[[4.26686, 4.266, 4.26686, 4.266, 4.26686, 4.2...",intermetallics_search_000458.cif,191,"{'Tb0+': 2.0, 'Fe0+': 14.144000000000005, 'Al0...","['Tb2(Fe0.832Al0.168)17-458-Cagome_big.gph', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,687536,687536,['Pt3Zr-687536-Cagome_big.gph'],intermetallics_search_687536.cif,intermetallics_search_687536.cif,[0],4,"[[1.0236380625, -1.0236380624999997, -1.023638...","[[6, 6, 6, 6]]",...,N/A,N/A,N/A,N/A,N/A,N/A,intermetallics_search_687536.cif,221,"{'Zr0+': 1.0, 'Pt0+': 3.0}",['Pt3Zr-687536-Cagome_big.gph']
14996,14996,687540,687540,"['CoVIn-687540-Cagome_big.gph', 'CoVIn-687540-...",intermetallics_search_687540.cif,intermetallics_search_687540.cif,"[0, 1]",12,"[[-1.841449, -1.8414489999999997, -1.841449, -...","[[6, 6, 6, 6], [6, 6, 6, 6]]",...,N/A,N/A,N/A,N/A,N/A,N/A,intermetallics_search_687540.cif,216,"{'V0+': 4.0, 'In0+': 4.0, 'Co0+': 4.0}","['CoVIn-687540-Cagome_big.gph', 'CoVIn-687540-..."
14997,14997,687555,687555,['GdCo2-687555-Cagome_big.gph'],intermetallics_search_687555.cif,intermetallics_search_687555.cif,[0],24,"[[0.034234250625, -0.03423425062499999, 0.0342...","[[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]",...,[4.53217],[0.0],[120.0],[0.0],"[[120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 12...","[[4.53217, 4.53217, 4.53217, 4.53217, 4.53217,...",intermetallics_search_687555.cif,227,"{'Gd0+': 8.0, 'Co0+': 16.0}",['GdCo2-687555-Cagome_big.gph']
14998,14998,687556,687556,['EuCo2-687556-Cagome_big.gph'],intermetallics_search_687556.cif,intermetallics_search_687556.cif,[0],24,"[[0.06837602, -0.06837601999999998, -0.0683760...","[[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]",...,[4.52911],[0.0],[120.0],[0.0],"[[120

In [778]:
data['cifs_ids'] = data['sorted_cifs_non_empty']

In [779]:
#data.to_csv('./bulk_1_all_grouped_multi_gph_20241021.csv')

In [785]:
# expand the list according to gph
data_expand = pd.DataFrame()

cifs_ids_expand = []
cifs_name_expand = []
space_group_num_expand = []
num_atom_expand = []
composition_expand = []

unique_gph_files_expand = []
plane_fun_gph_expand = []
CN_plane_expand = []
CN_group_index_expand = []
angle_labels_expand = []
distance_mean_60_expand = []
distance_std_60_expand = []
angles_mean_60_expand = []
angles_std_60_expand = []
angles_60_data_expand = []
distance_60_data_expand = []

distance_mean_120_expand = []
distance_std_120_expand = []
angles_mean_120_expand = []
angles_std_120_expand = []
angles_120_data_expand = []
distance_120_data_expand = []

unique_angles_expand = []
net_names_expand = []



for i in data.index:
    count_not_1 = 0 # this is used to index correctly the data for each label
    for gi in range(len(data['unique_gph'][i])):
        cifs_ids_expand.append(data['cifs_ids'][i])
        cifs_name_expand.append(data['cifs_name'][i])
        space_group_num_expand.append(data['space_group_num'][i])
        num_atom_expand.append(data['num_atom'][i])
        composition_expand.append(data['composition'][i])
        
        
        # check the unique_gph 
        unique_gph_index = data['unique_gph'][i][gi]
        
        gph_name = ast.literal_eval(data['gph_grouped'][i])[unique_gph_index]
        unique_gph_files_expand.append(gph_name)
        try:
            plane_fun_gph_expand.append(data['match_gph'][i][gi])
        except:
            plane_fun_gph_expand.append('N/A')
        if len(data['CN_group_index'][i])>0:
            if data['CN_group_index'][i][gi]==1: #most likely empty 


                net_names_expand.append('Likely_Empty')
                CN_plane_expand.append(data['CN_plane'][i][gi])
                CN_group_index_expand.append(data['CN_group_index'][i][gi])
                angle_labels_expand.append('Measured')

                distance_mean_60_expand.append(data['distance_mean_60'][i][gi-count_not_1])
                distance_std_60_expand.append(data['distance_std_60'][i][gi-count_not_1])
                angles_mean_60_expand.append(data['angles_mean_60'][i][gi-count_not_1])
                angles_std_60_expand.append(data['angles_std_60'][i][gi-count_not_1])
                angles_60_data_expand.append(data['angles_60_data'][i][gi-count_not_1])
                distance_60_data_expand.append(data['distance_60_data'][i][gi-count_not_1])

                distance_mean_120_expand.append(data['distance_mean_120'][i][gi-count_not_1])
                distance_std_120_expand.append(data['distance_std_120'][i][gi-count_not_1])
                angles_mean_120_expand.append(data['angles_mean_120'][i][gi-count_not_1])
                angles_std_120_expand.append(data['angles_std_120'][i][gi-count_not_1])
                angles_120_data_expand.append(data['angles_120_data'][i][gi-count_not_1])
                distance_120_data_expand.append(data['distance_120_data'][i][gi-count_not_1])

                unique_angles_expand.append(data['unique_angles'][i][gi-count_not_1])


            elif data['CN_group_index'][i][gi]==2: # most likely filled 


                net_names_expand.append('Likely_Filled')

                CN_plane_expand.append(data['CN_plane'][i][gi])
                CN_group_index_expand.append(data['CN_group_index'][i][gi])

                angle_labels_expand.append('Not measured')

                distance_mean_60_expand.append('N/A')
                distance_std_60_expand.append('N/A')
                angles_mean_60_expand.append('N/A')
                angles_std_60_expand.append('N/A')
                angles_60_data_expand.append('N/A')
                distance_60_data_expand.append('N/A')

                distance_mean_120_expand.append('N/A')
                distance_std_120_expand.append('N/A')
                angles_mean_120_expand.append('N/A')
                angles_std_120_expand.append('N/A')
                angles_120_data_expand.append('N/A')
                distance_120_data_expand.append('N/A')

                unique_angles_expand.append('N/A')
                count_not_1+=1

            else: # other nets with strange CN in-plane
                # basics


                net_names_expand.append('Other')            

                CN_plane_expand.append(data['CN_plane'][i][gi])
                CN_group_index_expand.append(data['CN_group_index'][i][gi])
                angle_labels_expand.append('Not measured')

                distance_mean_60_expand.append('N/A')
                distance_std_60_expand.append('N/A')
                angles_mean_60_expand.append('N/A')
                angles_std_60_expand.append('N/A')
                angles_60_data_expand.append('N/A')
                distance_60_data_expand.append('N/A')

                distance_mean_120_expand.append('N/A')
                distance_std_120_expand.append('N/A')
                angles_mean_120_expand.append('N/A')
                angles_std_120_expand.append('N/A')
                angles_120_data_expand.append('N/A')
                distance_120_data_expand.append('N/A')

                unique_angles_expand.append('N/A')
                count_not_1+=1
        else:
            net_names_expand.append('Other')            

            CN_plane_expand.append('N/A')
            CN_group_index_expand.append('N/A')
            angle_labels_expand.append('Not measured')

            distance_mean_60_expand.append('N/A')
            distance_std_60_expand.append('N/A')
            angles_mean_60_expand.append('N/A')
            angles_std_60_expand.append('N/A')
            angles_60_data_expand.append('N/A')
            distance_60_data_expand.append('N/A')

            distance_mean_120_expand.append('N/A')
            distance_std_120_expand.append('N/A')
            angles_mean_120_expand.append('N/A')
            angles_std_120_expand.append('N/A')
            angles_120_data_expand.append('N/A')
            distance_120_data_expand.append('N/A')

            unique_angles_expand.append('N/A')

In [786]:
data_expand['cifs_ids'] = cifs_ids_expand
data_expand['cifs_name'] = cifs_name_expand
data_expand['space_group_num'] = space_group_num_expand
data_expand['num_atom'] = num_atom_expand
data_expand['composition'] = composition_expand

data_expand['unique_gph_files'] = unique_gph_files_expand
data_expand['plane_fun_gph'] = plane_fun_gph_expand
data_expand['CN_plane'] = CN_plane_expand
data_expand['CN_group_index'] = CN_group_index_expand
data_expand['angle_labels'] = angle_labels_expand

data_expand['distance_mean_60'] = distance_mean_60_expand
data_expand['distance_std_60'] = distance_std_60_expand
data_expand['angles_mean_60'] = angles_mean_60_expand
data_expand['angles_std_60'] = angles_std_60_expand
data_expand['angles_60_data'] = angles_60_data_expand
data_expand['distance_60_data'] = distance_60_data_expand

data_expand['distance_mean_120'] = distance_mean_120_expand
data_expand['distance_std_120'] = distance_std_120_expand
data_expand['angles_mean_120'] = angles_mean_120_expand
data_expand['angles_std_120'] = angles_std_120_expand
data_expand['angles_120_data'] = angles_120_data_expand
data_expand['distance_120_data'] = distance_120_data_expand


data_expand['unique_angles'] = unique_angles_expand

data_expand['net_names'] = net_names_expand

In [ ]:
data_expand.to_csv('bulk_1_all_grouped_multi_gph_expand_20241021.csv')

In [792]:
# do not include structures with number of atoms lower than four. 
excluded_index = list(data_expand[data_expand['num_atom']<=4].index)

In [795]:
index_old = data[data['cifs_name'] == data_expand['cifs_name'][0]].index[0]

In [797]:
# analyze G-1 for inter-atomic distances and angles
G_2_angles = []
G_2_index = []
G_2_angle_distance_info_all = []

#data_all.iloc[G_1_index].index[:5]
p_bar = tqdm(data_expand.index[:])
for i in p_bar:
    measure_  = data_expand['angle_labels'][i]
    CN_i = data_expand['CN_plane'][i]

    nn_sites = None
    nn_info = None
    # only measure the structures that have yet been measured and have only coordination of 6 on the plane. 
    if i not in excluded_index and (6 in CN_i) and (len(np.unique(CN_i))==1): 
        #print(i)
        G_2_index.append(i)
        absds = data_expand['plane_fun_gph'][i]
        index_old_structure_list = data[data['cifs_name'] == data_expand['cifs_name'][i]].index[0]
        index_plane_structure = data['match_gph'][index_old_structure_list].index(absds)
        #print(index_old_structure_list,index_plane_structure)
        plane_structure = plane_structures_list[index_old_structure_list][index_plane_structure]
        plane_structure_pbc = plane_structures_pbc_list[index_old_structure_list][index_plane_structure]
        [A,B,C,D] = absds
        for cutoff in range(7,30):
            try:
                # Try to call your function with the current parameter
                C_NN = CrystalNN(distance_cutoffs=(2,2.4),x_diff_weight=0.0, porous_adjustment=False,search_cutoff=cutoff )
                nn_sites,nn_info = get_1nn_m(plane_structure_pbc,
                                   plane_structure,
                                   [A,B,C,D],
                                   NN_method=C_NN
                                  )
                cutoff_.append(cutoff)
                break
            except Exception as e:
                # If there is an error, print it and continue to the next parameter
                continue
        keys = nn_info.keys()
        G_2_angles.append(np.unique(np.concatenate([[x[3] for x in nn_info[k]['angle_info']]  for k in keys])))
        G_2_angle_distance_info_all.append([ nn_info[k]['angle_info'] for k in keys])
    else:
        G_2_angles.append('NA')
        G_2_angle_distance_info_all.append('NA')
        

  0%|          | 0/17780 [00:00<?, ?it/s]

In [800]:
G2_data_angles_around_60_mean = []
G2_data_angles_around_60_std = []
G2_data_djk_around_60_mean = []
G2_data_djk_around_60_std = []
G2_data_angles_around_60 = []
G2_data_djk_around_60 = []

# no data: 'NA'
for i in data_expand.index:
    if i in G_2_index:
        
        iG = i
        
        G_2_angle_distance_info_ = G_2_angle_distance_info_all[iG]
        #########
        test_di,test_angles = get_angles_distances_jik(G_2_angle_distance_info_)
        G2_angles_jik, G2_djk = get_quantity_std(test_di,test_angles,angles_around=60,delta=20)

        G2_data_djk_around_60_std.append(np.round(np.std(G2_djk),5))
        G2_data_angles_around_60_std.append(np.round(np.std(G2_angles_jik),5))
        G2_data_djk_around_60_mean.append(np.round(np.mean(G2_djk),5))
        G2_data_angles_around_60_mean.append(np.round(np.mean(G2_angles_jik),5))
        G2_data_angles_around_60.append(list(G2_angles_jik))
        G2_data_djk_around_60.append(list(G2_djk))
        ##########
        
    else:
        G2_data_djk_around_60_std.append('N/A')
        G2_data_angles_around_60_std.append('N/A')
        G2_data_djk_around_60_mean.append('N/A')
        G2_data_angles_around_60_mean.append('N/A')
        G2_data_angles_around_60.append('N/A')
        G2_data_djk_around_60.append('N/A')

In [802]:
G2_data_angles_around_120_mean = []
G2_data_angles_around_120_std = []
G2_data_djk_around_120_mean = []
G2_data_djk_around_120_std = []
G2_data_angles_around_120 = []
G2_data_djk_around_120 = []

# no data: 'NA'
for i in data_expand.index:
    if i in G_2_index:
        
        iG = i
        
        G_2_angle_distance_info_ = G_2_angle_distance_info_all[iG]
        #########
        test_di,test_angles = get_angles_distances_jik(G_2_angle_distance_info_)
        G2_angles_jik, G2_djk = get_quantity_std(test_di,test_angles,angles_around=120,delta=20)

        G2_data_djk_around_120_std.append(np.round(np.std(G2_djk),5))
        G2_data_angles_around_120_std.append(np.round(np.std(G2_angles_jik),5))
        G2_data_djk_around_120_mean.append(np.round(np.mean(G2_djk),5))
        G2_data_angles_around_120_mean.append(np.round(np.mean(G2_angles_jik),5))
        G2_data_angles_around_120.append(list(G2_angles_jik))
        G2_data_djk_around_120.append(list(G2_djk))
        ##########
        
    else:
        G2_data_djk_around_120_std.append('N/A')
        G2_data_angles_around_120_std.append('N/A')
        G2_data_djk_around_120_mean.append('N/A')
        G2_data_angles_around_120_mean.append('N/A')
        G2_data_angles_around_120.append('N/A')
        G2_data_djk_around_120.append('N/A')

In [804]:
angle_labels_expand_G2 = deepcopy(angle_labels_expand)
distance_mean_60_expand_G2 = deepcopy(distance_mean_60_expand)
distance_std_60_expand_G2 = deepcopy(distance_std_60_expand)
angles_mean_60_expand_G2 = deepcopy(angles_mean_60_expand)
angles_std_60_expand_G2 = deepcopy(angles_std_60_expand)
angles_60_data_expand_G2 = deepcopy(angles_60_data_expand)
distance_60_data_expand_G2 = deepcopy(distance_60_data_expand)
distance_mean_120_expand_G2 = deepcopy(distance_mean_120_expand)
distance_std_120_expand_G2 = deepcopy(distance_std_120_expand)
angles_mean_120_expand_G2 = deepcopy(angles_mean_120_expand)
angles_std_120_expand_G2 = deepcopy(angles_std_120_expand)
angles_120_data_expand_G2 = deepcopy(angles_120_data_expand)
distance_120_data_expand_G2 = deepcopy(distance_120_data_expand)

unique_angles_expand_G2 = deepcopy(unique_angles_expand)

In [806]:
for i in data_expand.index:
    if i in G_2_index:
        angle_labels_expand_G2[i] = "Measured"
        
        distance_mean_60_expand_G2[i] = G2_data_djk_around_60_mean[i] 
        distance_std_60_expand_G2[i] = G2_data_djk_around_60_std[i] 
        angles_mean_60_expand_G2[i] = G2_data_angles_around_60_mean[i]
        angles_std_60_expand_G2[i] = G2_data_angles_around_60_std[i]
        angles_60_data_expand_G2[i] = G2_data_angles_around_60[i]
        distance_60_data_expand_G2[i] = G2_data_djk_around_60[i]
        
        distance_mean_120_expand_G2[i] = G2_data_djk_around_120_mean[i]
        distance_std_120_expand_G2[i] = G2_data_djk_around_120_std[i]
        angles_mean_120_expand_G2[i] = G2_data_angles_around_120_mean[i]
        angles_std_120_expand_G2[i] = G2_data_angles_around_120_std[i]
        angles_120_data_expand_G2[i] = G2_data_angles_around_120[i]
        distance_120_data_expand_G2[i] = G2_data_djk_around_120[i]
        
        unique_angles_expand_G2[i] = list(np.unique(G_2_angles[i]))

In [807]:
data_expand['angle_labels'] = angle_labels_expand_G2

data_expand['distance_mean_60'] = distance_mean_60_expand_G2
data_expand['distance_std_60'] = distance_std_60_expand_G2
data_expand['angles_mean_60'] = angles_mean_60_expand_G2
data_expand['angles_std_60'] = angles_std_60_expand_G2
data_expand['angles_60_data'] = angles_60_data_expand_G2
data_expand['distance_60_data'] = distance_60_data_expand_G2

data_expand['distance_mean_120'] = distance_mean_120_expand_G2
data_expand['distance_std_120'] = distance_std_120_expand_G2
data_expand['angles_mean_120'] = angles_mean_120_expand_G2
data_expand['angles_std_120'] = angles_std_120_expand_G2
data_expand['angles_120_data'] = angles_120_data_expand_G2
data_expand['distance_120_data'] = distance_120_data_expand_G2

data_expand['unique_angles'] = unique_angles_expand_G2

In [809]:
data_expand.to_csv('./bulk_1_all_grouped_multi_gph_expand_2_20241111.csv')

In [635]:
# 
filtered_3atom_remain_index = [x for x in data.index if data['num_atom'][x]==3]
filtered_4atom_Cu3Au_type_remain_index = [x for x in data.index if (data['num_atom'][x]==4)&(data['space_group_num'][x]==221) ]
filtered_4atom_Cu3Au_type_remain_index_expand = [x for x in data.index if (data_expand['num_atom'][x]==4)&(data_expand['space_group_num'][x]==221) ]



In [639]:
Cu3Au_type_data = data.iloc[filtered_4atom_Cu3Au_type_remain_index]

In [641]:
Cu3Au_type_data.to_csv('All_Cu3Au_structure_type_perfect_filled.csv') 

In [636]:
data_expand.iloc[filtered_4atom_Cu3Au_type_remain_index_expand]

,cifs_ids,cifs_name,space_group_num,num_atom,composition,unique_gph_files,plane_fun_gph,CN_plane,CN_group_index,angle_labels,...,angles_60_data,distance_60_data,distance_mean_120,distance_std_120,angles_mean_120,angles_std_120,angles_120_data,distance_120_data,unique_angles,net_names
30,intermetallics_search_005116.cif,intermetallics_search_005116.cif,221,4,"{'Fe0+': 1.0, 'Ni0+': 3.0}",FeNi3-5116-Cagome_big.gph,"[0.8019202499999999, 0.8019202499999999, -0.80...","[6, 6, 6, 6]",2,Not measured,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Likely_Filled
81,intermetallics_search_012215.cif,intermetallics_search_012215.cif,221,4,"{'Dy0+': 1.0, 'Al0+': 3.0}",Al3Dy-12215-Cagome_big.gph,"[1.121481, -1.1214809999999997, -1.121481, 0.0]","[6, 6, 6, 6]",2,Not measured,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Likely_Filled
265,intermetallics_search_034328.cif,intermetallics_search_034328.cif,221,4,"{'Sr0+': 1.0, 'Sn0+': 0.75, 'Pb0+': 2.25}",SrSn0.75Pb2.25-34328-Cagome_big.gph,"[1.5342538224999998, -1.5342538224999998, 1.53...","[6, 6, 6, 6]",2,Not measured,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Likely_Filled
277,intermetallics_search_036719.cif,intermetallics_search_036719.cif,221,4,"{'Fe0+': 3.0, 'Ge0+': 1.0}",Fe3Ge-36719-Cagome_big.gph,"[-0.8402930556250001, 0.8402930556249999, 0.84...","[6, 6, 6, 6]",2,Not measured,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Likely_Filled
339,intermetallics_search_040334.cif,intermetallics_search_040334.cif,221,4,"{'Fe0+': 1.0, 'Ni0+': 3.0}",Ni3Fe-40334-Cagome_big.gph,"[-0.7854390624999998, 0.7854390624999998, -0.7...","[6, 6, 6, 6]",2,Not measured,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Likely_Filled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14989,intermetallics_search_648666.cif,intermetallics_search_648666.cif,221,4,"{'Yb0+': 1.0, 'Pb0+': 3.0}",Pb3Yb-648666-Cagome_big.gph,"[1.4779264899999998, -1.4779264899999998, 1.47...","[6, 6, 6, 6]",2,Not measured,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Likely_Filled
14991,intermetallics_search_648683.cif,intermetallics_search_648683.cif,221,4,"{'Pr0+': 1.0, 'Pd0+': 3.0}",Pd3Pr-648683-Cagome_big.gph,"[1.0718978556249998, -1.0718978556249998, 1.07...","[6, 6, 6, 6]",2,Not measured,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Likely_Filled
14992,intermetallics_search_648684.cif,intermetallics_search_648684.cif,221,4,"{'Pr0+': 1.0, 'Pd0+': 3.0}",Pd3Pr-648684-Cagome_big.gph,"[1.0705006225000002, -1.0705006225, -1.0705006...","[6, 6, 6, 6]",2,Not measured,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Likely_Filled
14993,intermetallics_search_648689.cif,intermetallics_search_648689.cif,221,4,"{'Pr0+': 1.0, 'Pd0+': 3.0}",Pd3Pr-648689-Cagome_big.gph,"[1.0686390624999997, -1.0686390624999997, 1.06...","[6, 6, 6, 6]",2,Not measured,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Likely_Filled
